In [44]:
import time
import numpy
import scipy.sparse
import scipy.sparse.linalg
# import pylab as pl
import pickle
import sys

In [45]:
import pyvista as pv
import vtk

In [46]:
from SILEX import silex_lib_tri3_acou
from SILEX import silex_lib_gmsh

In [47]:
#MPI/MUMPS
from mpi4py import MPI
comm = MPI.COMM_WORLD

nproc=comm.Get_size()
rank = comm.Get_rank()

import mumps

class comm_mumps_one_proc:
    rank = 0
    def py2f(self):
        return 0
mycomm=comm_mumps_one_proc()

In [48]:
##############################################################
##############################################################
#              S T A R T   M A I N   P R O B L E M
##############################################################
##############################################################

# parallepipedic cavity with plane structure
mesh_file='geom/xfem3'
results_file_ini='results/xfem3_'

##############################################################
# Material, Boundary conditions
##############################################################

# air
celerity=340.0
rho=1.2

freq_ini     = 100.0
freq_end     = 130.0
nb_freq_step_per_proc=5 # 50 pour 8 proc.

nproc=comm.Get_size()
rank = comm.Get_rank()

nb_freq_step = nb_freq_step_per_proc*nproc
deltafreq=(freq_end-freq_ini)/(nb_freq_step-1)

flag_write_gmsh_results=1

flag_edge_enrichment=0
#flag_edge_enrichment=1

#freq_comparaison = 210.0
press_para=list()
dpress_para=list()
        
h_struc=0.65
para_list= [8200,8201,8202]

for x_pos_struc_ini in para_list:

    file_extension=str(x_pos_struc_ini)[0:5]
    results_file=results_file_ini+file_extension

    x_pos_struc=x_pos_struc_ini/10000

    ##############################################################
    # Load fluid mesh
    ##############################################################

    tic = time.process_time()

    fluid_nodes    = silex_lib_gmsh.ReadGmshNodes(mesh_file+'_fluid.msh',2)
    fluid_elements,Idnodes = silex_lib_gmsh.ReadGmshElements(mesh_file+'_fluid.msh',2,1)
    fluid_elements5,Idnodes5 = silex_lib_gmsh.ReadGmshElements(mesh_file+'_fluid.msh',2,5)

    fluid_nnodes   = fluid_nodes.shape[0]
    fluid_nelem    = fluid_elements.shape[0]
    fluid_ndof     = fluid_nnodes

    fluid_elements_boun,IdnodeS2 = silex_lib_gmsh.ReadGmshElements(mesh_file+'_fluid.msh',1,2)

    if (flag_write_gmsh_results==1) and (rank==0):
        silex_lib_gmsh.WriteResults(results_file+'_fluid_mesh',fluid_nodes,fluid_elements,2)
        silex_lib_gmsh.WriteResults(results_file+'_control_volume_fluid_mesh',fluid_nodes,fluid_elements5,2)
        silex_lib_gmsh.WriteResults(results_file+'_fluid_boundary',fluid_nodes,fluid_elements_boun,1)
    print("nnodes for fluid=",fluid_nnodes)
    print("nelem for fluid=",fluid_nelem)



    ##################################################################
    # compute level set
    ##################################################################

    tic = time.process_time()

    LevelSet=fluid_nodes[:,0]-x_pos_struc
    LevelSetTangent=fluid_nodes[:,1]-h_struc

    # level set gradient with respect to parameters
    LevelSet_gradient=-numpy.ones(fluid_nnodes)
    LevelSetTangent_gradient=numpy.zeros(fluid_nnodes)

    if (flag_write_gmsh_results==1) and (rank==0):
        silex_lib_gmsh.WriteResults(results_file+'_tangent_level_set',fluid_nodes,fluid_elements,2,[[LevelSetTangent,'nodal',1,'Tangent level set']])
        silex_lib_gmsh.WriteResults(results_file+'_level_set',fluid_nodes,fluid_elements,2,[[LevelSet,'nodal',1,'Level set']])

    toc = time.process_time()
    print("time to compute level set:",toc-tic)


    ##################################################################
    # Get enriched nodes and elements
    ##################################################################
    tic = time.process_time()

    struc_nodes=numpy.array([[x_pos_struc,0.0],[x_pos_struc,h_struc/2.0],[x_pos_struc,h_struc]])
    struc_elements=numpy.array([[1,2],[2,3]])
    struc_boun=numpy.array([3])

    silex_lib_gmsh.WriteResults(results_file+'_struc_mesh',struc_nodes,struc_elements,1)

    EnrichedElements,NbEnrichedElements=silex_lib_tri3_acou.getenrichedelements(struc_nodes,struc_elements,fluid_nodes,fluid_elements)
    EnrichedElements=numpy.unique(EnrichedElements[list(range(NbEnrichedElements))])-1

    toc = time.process_time()
    print("time to find surface enriched elements:",toc-tic)


    tic = time.process_time()

    EdgeEnrichedElements,nbenrelts = silex_lib_tri3_acou.getedgeenrichedelements(struc_nodes,struc_boun,fluid_nodes,fluid_elements)
    EdgeEnrichedElements=numpy.unique(EdgeEnrichedElements[list(range(nbenrelts))])-1

    #EnrichedElements=numpy.setdiff1d(EnrichedElements,EdgeEnrichedElements)

    toc = time.process_time()
    print("time to find edge enriched elements:",toc-tic)

    if (flag_write_gmsh_results==1) and (rank==0):
        silex_lib_gmsh.WriteResults(results_file+'_enriched_elements',fluid_nodes,fluid_elements[EnrichedElements],2)
    if (flag_write_gmsh_results==1) and (rank==0):
        silex_lib_gmsh.WriteResults(results_file+'_edge_enriched_elements',fluid_nodes,fluid_elements[EdgeEnrichedElements],2)

    ##############################################################
    # Compute Standard Fluid Matrices
    ##############################################################
    tic = time.process_time()

    IIf,JJf,Vffk,Vffm=silex_lib_tri3_acou.globalacousticmatrices(fluid_elements,fluid_nodes,celerity,rho)

    KFF = scipy.sparse.csc_matrix( (Vffk,(IIf,JJf)), shape=(fluid_ndof,fluid_ndof) )
    MFF = scipy.sparse.csc_matrix( (Vffm,(IIf,JJf)), shape=(fluid_ndof,fluid_ndof) )

    SolvedDofF=numpy.setdiff1d(list(range(fluid_ndof)),IdnodeS2-1)
    #SolvedDofF=range(fluid_ndof)

    toc = time.process_time()
    print("time to compute fluid matrices:",toc-tic)

    ##################################################################
    # Compute enrichment: Heaviside + Edge
    ##################################################################
    tic = time.process_time()

    HeavisideEnrichedElements=numpy.setdiff1d(EnrichedElements,EdgeEnrichedElements)

    #Enrichednodes = numpy.unique(fluid_elements[HeavisideEnrichedElements])
    #Enrichednodes = numpy.unique(fluid_elements[EnrichedElements])

    #print xvibacoufo.getpositivenegativeelts.__doc__

    NegativeLSelements,PositiveLSelements,NegativeLStgtElements,PositiveLStgtElements,nbNegLS,nbPosLS,nbNegLSt,nbPosLSt=silex_lib_tri3_acou.getpositivenegativeelts(fluid_elements,LevelSet,LevelSetTangent)

    NegativeLSelements=NegativeLSelements[list(range(nbNegLS))]
    PositiveLSelements=PositiveLSelements[list(range(nbPosLS))]
    NegativeLStgtElements=NegativeLStgtElements[list(range(nbNegLSt))]
    PositiveLStgtElements=PositiveLStgtElements[list(range(nbPosLSt))]

    EdgeEnrichedElementsInAllMesh,nbEdgeEnrichedElementsInAllMesh=silex_lib_tri3_acou.getenrichedelementsfromlevelset(fluid_elements,LevelSetTangent)
    EdgeEnrichedElementsInAllMesh=numpy.unique(EdgeEnrichedElementsInAllMesh[list(range(nbEdgeEnrichedElementsInAllMesh))])
    
    IdElementTip=silex_lib_tri3_acou.getelementcontainingpoint(fluid_elements,fluid_nodes,[0.6,0.65])

    #if (flag_write_gmsh_results==1) and (rank==0):
    #    silex_lib_gmsh.WriteResults(results_file+'_NegativeLSelements',fluid_nodes,fluid_elements[NegativeLSelements],2)
    #    silex_lib_gmsh.WriteResults(results_file+'_PositiveLSelements',fluid_nodes,fluid_elements[PositiveLSelements],2)
    #    silex_lib_gmsh.WriteResults(results_file+'_NegativeLStgtElements',fluid_nodes,fluid_elements[NegativeLStgtElements],2)
    #    silex_lib_gmsh.WriteResults(results_file+'_PositiveLStgtElements',fluid_nodes,fluid_elements[PositiveLStgtElements],2)




    IIaa,JJaa,IIaf,JJaf,Vaak,Vaam,Vafk,Vafm=silex_lib_tri3_acou.globalxfemacousticmatrices(fluid_elements,fluid_nodes,LevelSet,LevelSetTangent,celerity,rho,flag_edge_enrichment)

    KAA = scipy.sparse.csc_matrix( (Vaak,(IIaa,JJaa)), shape=(fluid_ndof,fluid_ndof) )
    MAA = scipy.sparse.csc_matrix( (Vaam,(IIaa,JJaa)), shape=(fluid_ndof,fluid_ndof) )
    KAF = scipy.sparse.csc_matrix( (Vafk,(IIaf,JJaf)), shape=(fluid_ndof,fluid_ndof) )
    MAF = scipy.sparse.csc_matrix( (Vafm,(IIaf,JJaf)), shape=(fluid_ndof,fluid_ndof) )

    toc = time.process_time()
    print("time to compute Heaviside enrichment:",toc-tic)

    #Enrichednodes = numpy.unique(fluid_elements[numpy.hstack(([HeavisideEnrichedElements,EdgeEnrichedElements]))])
    #Enrichednodes = numpy.unique(fluid_elements[numpy.hstack(([EnrichedElements,PositiveLStgtElements,EdgeEnrichedElementsInAllMesh]))])
    #Enrichednodes = numpy.unique(fluid_elements[numpy.hstack(([EnrichedElements,PositiveLStgtElements]))])
    #Enrichednodes = numpy.unique(fluid_elements[numpy.hstack(([NegativeLStgtElements]))])
    Enrichednodes = numpy.unique(fluid_elements[EnrichedElements])
    #Enrichednodes = numpy.unique(fluid_elements)

    silex_lib_gmsh.WriteResults(results_file+'_EnrichedElements',fluid_nodes,fluid_elements[numpy.hstack(([EnrichedElements]))],2)
    #silex_lib_gmsh.WriteResults(results_file+'PositiveLStgtElements',fluid_nodes,fluid_elements[numpy.hstack(([PositiveLStgtElements]))],2)
    #silex_lib_gmsh.WriteResults(results_file+'EdgeEnrichedElementsInAllMesh',fluid_nodes,fluid_elements[numpy.hstack(([EdgeEnrichedElementsInAllMesh]))],2)

    #EdgeEnrichedEltsTest=numpy.hstack(([PositiveLStgtElements,EdgeEnrichedElementsInAllMesh]))
    #silex_lib_gmsh.WriteResults(results_file+'EdgeEnrichedEltsTest',fluid_nodes,fluid_elements[numpy.hstack(([EdgeEnrichedEltsTest]))],2)
    #EdgeEnrichednodesTest = numpy.unique(fluid_elements[EdgeEnrichedEltsTest])
    #Enrichednodes = numpy.unique(numpy.hstack(([Enrichednodes,EdgeEnrichednodesTest])))

    SolvedDofA=Enrichednodes-1


    #################################################################
    # Construct the whole system
    ##################################################################

    K=scipy.sparse.construct.bmat( [[KFF[SolvedDofF,:][:,SolvedDofF],KAF[SolvedDofF,:][:,SolvedDofA]],
                                    [KAF[SolvedDofA,:][:,SolvedDofF],KAA[SolvedDofA,:][:,SolvedDofA]]
                                    ] )



    M=scipy.sparse.construct.bmat( [[MFF[SolvedDofF,:][:,SolvedDofF],MAF[SolvedDofF,:][:,SolvedDofA]],
                                    [MAF[SolvedDofA,:][:,SolvedDofF],MAA[SolvedDofA,:][:,SolvedDofA]]
                                    ] )

    #################################################################
    # Compute gradients with respect to parameters
    ##################################################################

    #print(silex_lib_tri3_acou.globalacousticgradientmatrices.__doc__)

    IIf,JJf,Vfak_gradient,Vfam_gradient,PtsIntegration,nbPtsintegration=silex_lib_tri3_acou.globalacousticgradientmatrices(fluid_elements[EnrichedElements],fluid_nodes,celerity,rho,LevelSet_gradient,LevelSet,LevelSetTangent)
    #IIf,JJf,Vfak_gradient,Vfam_gradient=silex_lib_tri3_acou.globalacousticgradientmatrices(fluid_elements,fluid_nodes,celerity,rho,LevelSet_gradient,LevelSet)
    dMFA_dtheta = scipy.sparse.csc_matrix( (Vfam_gradient,(IIf,JJf)), shape=(fluid_ndof,fluid_ndof) )
    dKFA_dtheta = scipy.sparse.csc_matrix( (Vfak_gradient,(IIf,JJf)), shape=(fluid_ndof,fluid_ndof) )
    PtsIntegration=PtsIntegration[list(range(nbPtsintegration)),:]

    print(silex_lib_tri3_acou.writepointsongmshfile.__doc__)
    silex_lib_tri3_acou.writepointsongmshfile('PtsIntegrationStructureGradient.gmsh',PtsIntegration)
    
    #print(silex_lib_tri3_acou.edgegradientglobalxfemacousticmatrices.__doc__)
    IIaa,JJaa,IIaf,JJaf,Vaak_edge_gradient,Vaam_edge_gradient,Vafk_edge_gradient,Vafm_edge_gradient = silex_lib_tri3_acou.edgegradientglobalxfemacousticmatrices(fluid_elements,fluid_nodes,LevelSet,LevelSetTangent,LevelSet_gradient,LevelSetTangent_gradient,celerity,rho,flag_edge_enrichment)
    dKAA_edge_dtheta = scipy.sparse.csc_matrix( (Vaak_edge_gradient,(IIaa,JJaa)), shape=(fluid_ndof,fluid_ndof) )
    dKAF_edge_dtheta = scipy.sparse.csc_matrix( (Vafk_edge_gradient,(IIaf,JJaf)), shape=(fluid_ndof,fluid_ndof) )
    dMAA_edge_dtheta = scipy.sparse.csc_matrix( (Vaam_edge_gradient,(IIaa,JJaa)), shape=(fluid_ndof,fluid_ndof) )
    dMAF_edge_dtheta = scipy.sparse.csc_matrix( (Vafm_edge_gradient,(IIaf,JJaf)), shape=(fluid_ndof,fluid_ndof) )


    f=open(results_file+'_KAF_MAF.pck','wb')
    pickle.dump([dKFA_dtheta[SolvedDofF,:][:,SolvedDofA],dMFA_dtheta[SolvedDofF,:][:,SolvedDofA],KAF[SolvedDofF,:][:,SolvedDofA],MAF[SolvedDofF,:][:,SolvedDofA]], f)
    f.close()

##    dK=scipy.sparse.construct.bmat( [[None,dKFA_dtheta[SolvedDofF,:][:,SolvedDofA]],
##                                     [dKFA_dtheta[SolvedDofA,:][:,SolvedDofF],None]
##                                    ] )
##
##    dM=scipy.sparse.construct.bmat( [[None,dMFA_dtheta[SolvedDofF,:][:,SolvedDofA]],
##                                     [dMFA_dtheta[SolvedDofA,:][:,SolvedDofF],None]
##                                    ] )
    
    dK=scipy.sparse.construct.bmat( [[None,dKFA_dtheta[SolvedDofF,:][:,SolvedDofA]+dKAF_edge_dtheta[SolvedDofF,:][:,SolvedDofA]],
                                     [dKFA_dtheta[SolvedDofA,:][:,SolvedDofF]+dKAF_edge_dtheta[SolvedDofA,:][:,SolvedDofF],dKAA_edge_dtheta[SolvedDofA,:][:,SolvedDofA]]
                                    ] )

    dM=scipy.sparse.construct.bmat( [[None,dMFA_dtheta[SolvedDofF,:][:,SolvedDofA]+dMAF_edge_dtheta[SolvedDofF,:][:,SolvedDofA]],
                                     [dMFA_dtheta[SolvedDofA,:][:,SolvedDofF]+dMAF_edge_dtheta[SolvedDofA,:][:,SolvedDofF],dMAA_edge_dtheta[SolvedDofA,:][:,SolvedDofA]]
                                    ] )

    ##############################################################
    # FRF computation of the FSI problem
    ##############################################################

    Flag_frf_analysis=1
    FF = numpy.zeros(fluid_ndof)
    frequencies=[]
    frf=[]
    frfgradient=[]

    if (Flag_frf_analysis==1):
        print("time at the beginning of the FRF:",time.ctime())

        press_save=[]
        dpress_save=[]
        disp_save=[]

        for i in range(nb_freq_step_per_proc):

            freq = freq_ini+i*nproc*deltafreq+rank*deltafreq
            frequencies.append(freq)
            omega=2*scipy.pi*freq
            print("proc number",rank,"frequency=",freq)

            FF[SolvedDofF]=-(KFF[SolvedDofF,:][:,IdnodeS2-1]-(omega**2)*MFF[SolvedDofF,:][:,IdnodeS2-1])*(numpy.ones((len(IdnodeS2))))
            FA = numpy.zeros(fluid_ndof)
            F  = FF[SolvedDofF]
            F  = numpy.append(F,FA[SolvedDofA])
            #F  = scipy.sparse.csc_matrix(F)

            #sol = mumps.spsolve(scipy.sparse.coo_matrix(K-(omega**2)*M,dtype='float'), F, comm=mycomm )
            sol = scipy.sparse.linalg.spsolve(scipy.sparse.csc_matrix(K-(omega**2)*M,dtype='float'), F)
            
            #eigen_values_small,eigen_vectors= scipy.sparse.linalg.eigsh(K-(omega**2)*M,1,sigma=0,which='SM')
            #eigen_values_large,eigen_vectors= scipy.sparse.linalg.eigsh(K-(omega**2)*M,1,sigma=0,which='LM')


            #eigen_values_small,eigen_vectors= scipy.linalg.eig((K-(omega**2)*M).todense(),1,sigma=0,which='SM')
            #eigen_values_large,eigen_vectors= scipy.linalg.eig(K-(omega**2)*M,1,sigma=0,which='LM')

            
            #stop
            tmp=-(dK-(omega**2)*dM)*sol

            #Dsol_Dtheta = mumps.spsolve(  scipy.sparse.coo_matrix(K-(omega**2)*M,dtype='float')  , tmp , comm=mycomm )
            Dsol_Dtheta = scipy.sparse.linalg.spsolve(  scipy.sparse.csc_matrix(K-(omega**2)*M,dtype='float')  , tmp )

            press = numpy.zeros(fluid_ndof)
            press[IdnodeS2-1] = numpy.ones(len(IdnodeS2))
            press[SolvedDofF]=sol[list(range(len(SolvedDofF)))]
            enrichment=numpy.zeros(fluid_nnodes)
            enrichment[SolvedDofA]=sol[list(range(len(SolvedDofF),len(SolvedDofF)+len(SolvedDofA)))]
            CorrectedPressure=press
            CorrectedPressure[SolvedDofA]=CorrectedPressure[SolvedDofA]+enrichment[SolvedDofA]*numpy.sign(LevelSet[SolvedDofA])
            #frf.append(silex_acou_lib_tri3.computequadratiquepressure(fluid_elements,fluid_nodes,CorrectedPressure))
            frf.append(silex_lib_tri3_acou.computexfemcomplexquadratiquepressure(fluid_elements5,fluid_nodes,press+0j,0.0*enrichment+0j,LevelSet,LevelSetTangent,flag_edge_enrichment))
            #frf[i]=xvibacoufo.computexfemcomplexquadratiquepressure(fluid_elements,fluid_nodes,CorrectedPressure+0j,0.0*enrichment+0j,LevelSet,LevelSetTangent)
            #press_save.append(CorrectedPressure.copy())
            press_save.append(press.copy())

            Dpress_Dtheta = numpy.zeros(fluid_ndof,dtype=float)
            Dpress_Dtheta[SolvedDofF] = Dsol_Dtheta[list(range(len(SolvedDofF)))]
            Denrichment_Dtheta = numpy.zeros(fluid_ndof,dtype=float)
            Denrichment_Dtheta[SolvedDofA]= Dsol_Dtheta[list(range(len(SolvedDofF),len(SolvedDofF)+len(SolvedDofA)))]
            #DCorrectedPressure_Dtheta=numpy.array(Dpress_Dtheta)
            #DCorrectedPressure_Dtheta[SolvedDofA]=DCorrectedPressure_Dtheta[SolvedDofA].T+numpy.array(Denrichment_Dtheta[SolvedDofA]*numpy.sign(LevelSet[SolvedDofA]).T)
            frfgradient.append(silex_lib_tri3_acou.computexfemcomplexquadratiquepressuregradient(fluid_elements5,fluid_nodes,press+0j,Dpress_Dtheta+0j,0.0*enrichment+0j,0.0*Denrichment_Dtheta+0j,LevelSet,LevelSetTangent,flag_edge_enrichment))
            dpress_save.append(Dpress_Dtheta.copy())
        

        print("time at the end of the FRF:",time.ctime())
        frfsave=[frequencies,frf,frfgradient]
        if rank!=0:
            comm.send(frfsave, dest=0, tag=11)
        if (flag_write_gmsh_results==1) and (rank==0):
            silex_lib_gmsh.WriteResults2(results_file+'_results_fluid_frf',fluid_nodes,fluid_elements,2,[[press_save,'nodal',1,'pressure'],[dpress_save,'nodal',1,'pressure gradient']])

        #save pressure
        press_para.append(press_save)
        dpress_para.append(dpress_save)

        # save the FRF problem
        Allfrequencies=numpy.zeros(nb_freq_step)
        Allfrf=numpy.zeros(nb_freq_step)
        Allfrfgradient=numpy.zeros(nb_freq_step)
        k=0
        if rank==0:
            for i in range(nproc):
                if i==0:
                    data = frfsave
                else:
                    data = comm.recv(source=i, tag=11)
                for j in range(len(data[0])):
                    Allfrequencies[k]=data[0][j]
                    Allfrf[k]=data[1][j]
                    Allfrfgradient[k]=data[2][j]
                    k=k+1

            Allfrequencies, Allfrf,Allfrfgradient = zip(*sorted(zip(Allfrequencies, Allfrf,Allfrfgradient)))
            Allfrfsave=[numpy.array(list(Allfrequencies)),numpy.array(list(Allfrf)),numpy.array(list(Allfrfgradient)),press_save,dpress_save]
            f=open(results_file+'_results.frf','wb')
            pickle.dump(Allfrfsave, f)
            f.close()




nnodes for fluid= 971
nelem for fluid= 1814
time to compute level set: 0.013635999999999981
time to find surface enriched elements: 0.0009259999999997603
time to find edge enriched elements: 7.599999999996498e-05
time to compute fluid matrices: 0.0019080000000002428
time to compute Heaviside enrichment: 0.004865999999999815
writepointsongmshfile(filename,points,[nbpoints])

Wrapper for ``writepointsongmshfile``.

Parameters
----------
filename : input string(len=-1)
points : input rank-2 array('d') with bounds (nbpoints,2)

Other Parameters
----------------
nbpoints : input int, optional
    Default: shape(points,0)

time at the beginning of the FRF: Wed Dec  8 17:50:42 2021
proc number 0 frequency= 100.0
 pquadelem    8.7081196233700553E-004
 ve    5.9685221809376823E-004
proc number  pquadelem    9.2264407302261511E-004
 ve    5.9685221809378927E-004
 pquadelem    8.0352709072794063E-004
 ve    5.9685221809377528E-004
 pquadelem    8.3555770185625216E-004
 ve    5.9685221809378927E-0

In [49]:
fluid_elements5

array([[ 77, 208, 193],
       [ 72, 210, 194],
       [ 82, 209, 192],
       [ 87, 211, 195],
       [ 82, 192, 204],
       [ 87, 195, 205],
       [ 77, 193, 206],
       [ 72, 194, 207],
       [192, 200, 204],
       [195, 201, 205],
       [188, 204, 200],
       [189, 205, 201],
       [193, 202, 206],
       [194, 203, 207],
       [190, 206, 202],
       [191, 207, 203],
       [ 77,  78, 208],
       [ 72,  73, 210],
       [ 82,  83, 209],
       [ 87,  88, 211],
       [ 81,  82, 204],
       [ 86,  87, 205],
       [ 76,  77, 206],
       [ 71,  72, 207],
       [188, 213, 208],
       [189, 212, 209],
       [190, 215, 210],
       [191, 214, 211],
       [208, 216, 188],
       [209, 217, 189],
       [210, 218, 190],
       [204, 188, 220],
       [205, 189, 221],
       [211, 219, 191],
       [206, 190, 222],
       [207, 191, 223],
       [216, 197, 188],
       [217, 196, 189],
       [218, 199, 190],
       [219, 198, 191],
       [220, 188, 197],
       [221, 189

In [50]:
frf


[1.4339168874277683,
 3.007486940301543,
 8.82180685220548,
 175.40337380656976,
 17.648890496923375]

In [51]:
import pyvista as pv
import vtk

In [52]:
numnodes=fluid_elements.shape[1]*numpy.ones(fluid_elements.shape[0],dtype=numpy.uint8)[:,numpy.newaxis]
cells=numpy.hstack([numnodes,fluid_elements-1])
celltypes = numpy.empty(fluid_elements.shape[0],dtype = numpy.uint8)
celltypes[:] = vtk.VTK_TRIANGLE
nodes=numpy.hstack((fluid_nodes,numpy.zeros((fluid_nodes.shape[0],1))))
grid = pv.UnstructuredGrid(cells.ravel(),celltypes,nodes)

# grid.plot()

In [53]:
itfreq = 3
itpara = 1

grid['Freq {}'.format(frequencies[i])] = press_para[itpara][itfreq]
#grid.plot()
grid['deriv Freq {}'.format(frequencies[i])] = dpress_para[itpara][itfreq]
# grid.plot(jupyter_backend='pythreejs')
plotter = pv.Plotter()
plotter.add_mesh(grid)
plotter.show(jupyter_backend='ipygany')


AppLayout(children=(VBox(children=(HTML(value='<h3>deriv Freq 100.0</h3>'), Dropdown(description='Colormap:', …

In [54]:
deltaX = (para_list[itpara+1]-para_list[itpara-1])/10000
deltaX

0.0002

In [55]:
grid['deriv CD {}'.format(frequencies[i])] = (press_para[itpara+1][itfreq]-press_para[itpara-1][itfreq])/deltaX
plotter.add_mesh(grid)
plotter.show(jupyter_backend='ipygany')


AppLayout(children=(VBox(children=(HTML(value='<h3>deriv CD 100.0</h3>'), Dropdown(description='Colormap:', op…

In [56]:
plotter = pv.PlotterITK(notebook=True)
gridtmp[nametxt] = press_para[i][j]
#plotter.add_mesh(grid,smooth_shading=False)
# plotter.show()

NameError: name 'gridtmp' is not defined

In [ ]:
for i in range(len(para_list)):
    for j in range(len(press_para[i])):
        gridtmp = grid.copy()
        nametxt = 'Para: {} - Pressure freq. {}'.format(para_list[i],frequencies[j])
        gridtmp[nametxt] = press_para[i][j]
        plotter.add_mesh(gridtmp,smooth_shading=False)
        gridtmp = grid.copy()
        nametxt = 'Para: {} - Deriv. pressure freq. {}'.format(para_list[i],frequencies[j])
        gridtmp[nametxt] = dpress_para[i][j]
        plotter.add_mesh(gridtmp,smooth_shading=False)
plotter.show(True)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [ ]:
# create a mesh and identify some scalars you wish to plot
mesh = pv.Sphere()
z = mesh.points[:, 2]

# Plot using the itkwidget
pv.plot_itk(mesh, scalars=z)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [ ]:
# grid.plot(jupyter_backend='pythreejs')
grid

UnstructuredGrid,Information
N Cells,242
N Points,142
X Bounds,"0.000e+00, 1.000e+00"
Y Bounds,"0.000e+00, 1.000e+00"
Z Bounds,"0.000e+00, 0.000e+00"
N Arrays,0


In [ ]:
grid['ou']=press_para[0][0]
plotter.add_mesh(grid)
plotter.show(True)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [ ]:
plotter.

SyntaxError: invalid syntax (3508497951.py, line 1)